In [ ]:
# Install dependencies (if not already installed)
!pip install lxml beautifulsoup4 tqdm

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import time
from tqdm import tqdm
from google.colab import files

# --- Utility Functions ---

def scrape_match(match_id):
    """
    Scrapes shot data from a single match on Understat.
    Returns a DataFrame if successful, or None if not.
    """
    url = f"https://understat.com/match/{match_id}"
    try:
        res = requests.get(url)
        # Skip if the page isn't found
        if res.status_code != 200:
            print(f"Match {match_id} not found (status code {res.status_code}).")
            return None

        soup = BeautifulSoup(res.content, 'lxml')
        scripts = soup.find_all('script')
        # Check if the script that contains JSON data is available
        if len(scripts) < 2 or scripts[1].string is None:
            print(f"Match {match_id}: Could not find the proper script element.")
            return None

        strings = scripts[1].string
        # Extract the JSON data using the known pattern
        ind_start = strings.index("('") + 2
        ind_end = strings.index("')")
        json_data = strings[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')
        data = json.loads(json_data)

        # Create DataFrames for home and away shot data and add a match_id column
        df_h = pd.DataFrame(data.get('h', []))
        df_a = pd.DataFrame(data.get('a', []))
        if df_h.empty and df_a.empty:
            return None

        df = pd.concat([df_h, df_a], ignore_index=True)
        df['match_id'] = match_id
        return df
    except Exception as e:
        print(f"Error processing match {match_id}: {e}")
        return None

# --- Main Loop for Season Scraping ---

# Set the range of match IDs to scrape.
# Adjust these numbers if needed for the specific season.
start_id = 21925
end_id = 22305  # This will loop through match IDs 22200 to 22299

all_matches = []  # List to store DataFrames from each match

print("Starting to scrape matches...")
for match_id in tqdm(range(start_id, end_id), desc="Processing Matches"):
    df_match = scrape_match(match_id)
    if df_match is not None:
        all_matches.append(df_match)
    time.sleep(0.5)  # Optional: slow down requests to be polite to the server

# Combine all the match DataFrames if any data was collected
if all_matches:
    final_df = pd.concat(all_matches, ignore_index=True)
    output_path = '/content/EPL_2023_24_all_matches_shots.csv'
    final_df.to_csv(output_path, index=False)
    print(f"\n✅ CSV file successfully saved at: {output_path}")
    # Auto-download the file to your local machine
    files.download(output_path)
else:
    print("⚠️ No match data was successfully scraped.")


Starting to scrape matches...


Processing Matches:  34%|███▍      | 130/380 [02:22<04:29,  1.08s/it]

Match 22055 not found (status code 404).


Processing Matches: 100%|██████████| 380/380 [06:55<00:00,  1.09s/it]



✅ CSV file successfully saved at: /content/EPL_2023_24_all_matches_shots.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Scraping current season's data:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from google.colab import files

# Step 1: Upload your existing CSV to Colab first manually or with files.upload()
uploaded = files.upload()

# Replace with the name of your uploaded file
existing_file = "EPL_2023-24_all_matches_shots.csv"

# Step 2: Load existing data
try:
    df_existing = pd.read_csv(existing_file)
    print(f"✅ Existing data loaded: {df_existing.shape[0]} rows")
except FileNotFoundError:
    df_existing = pd.DataFrame()
    print("⚠️ Existing file not found. Starting with empty DataFrame.")

# Step 3: Define scraper for individual match
def scrape_match(match_id):
    try:
        url = f"https://understat.com/match/{match_id}"
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        scripts = soup.find_all('script')
        strings = scripts[1].string
        ind_start = strings.index("('") + 2
        ind_end = strings.index("')")
        json_data = strings[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')
        data = json.loads(json_data)
        df_h = pd.DataFrame(data['h'])
        df_a = pd.DataFrame(data['a'])
        df = pd.concat([df_h, df_a], ignore_index=True)
        return df
    except Exception as e:
        print(f"⚠️ Error processing match {match_id}: {e}")
        return None

# Step 4: Scrape new matches (26602–26908)
new_matches = []

for match_id in range(26602, 26909):
    print(f"🔍 Scraping match {match_id}...")
    df_match = scrape_match(match_id)
    if df_match is not None and not df_match.empty:
        new_matches.append(df_match)

# Step 5: Combine new data
if new_matches:
    df_new = pd.concat(new_matches, ignore_index=True)
    print(f"✅ New matches scraped: {df_new.shape[0]} rows")
else:
    print("⚠️ No new match data found.")
    df_new = pd.DataFrame()

# Step 6: Append new data to existing
if not df_new.empty:
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    output_file = "EPL_2023-24_all_matches_shots_UPDATED.csv"
    df_combined.to_csv(output_file, index=False)
    print(f"✅ Combined data saved! Total rows: {df_combined.shape[0]}")

    # Step 7: Download the updated file to your local machine
    files.download(output_file)
else:
    print("⚠️ No new data to append. Existing file remains unchanged.")


Saving EPL_2023-24_all_matches_shots.csv to EPL_2023-24_all_matches_shots.csv
✅ Existing data loaded: 10492 rows
🔍 Scraping match 26602...
🔍 Scraping match 26603...
🔍 Scraping match 26604...
🔍 Scraping match 26605...
🔍 Scraping match 26606...
🔍 Scraping match 26607...
🔍 Scraping match 26608...
🔍 Scraping match 26609...
🔍 Scraping match 26610...
🔍 Scraping match 26611...
🔍 Scraping match 26612...
🔍 Scraping match 26613...
🔍 Scraping match 26614...
🔍 Scraping match 26615...
🔍 Scraping match 26616...
🔍 Scraping match 26617...
🔍 Scraping match 26618...
🔍 Scraping match 26619...
🔍 Scraping match 26620...
🔍 Scraping match 26621...
🔍 Scraping match 26622...
🔍 Scraping match 26623...
🔍 Scraping match 26624...
🔍 Scraping match 26625...
🔍 Scraping match 26626...
🔍 Scraping match 26627...
🔍 Scraping match 26628...
🔍 Scraping match 26629...
🔍 Scraping match 26630...
🔍 Scraping match 26631...
🔍 Scraping match 26632...
🔍 Scraping match 26633...
🔍 Scraping match 26634...
🔍 Scraping match 26635...
🔍 S

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>